# Optimize Prompting Technique

## Setup

In [ ]:
!pip3 install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
API_KEY = ''

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## Zero-shot prompting

In [ ]:
prompt = '''
Classify the text into neutral, negative or positive.
Text: I think the vacation is okay.
Sentiment:'''

response = get_completion(prompt)
print(response)

## Few-shot prompting

In [ ]:
prompt='''
A "whatpu" is a small, furry animal native to Tanzania.
An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.

To do a "farduddle" means to jump up and down really fast.
An example of a sentence that uses the word farduddle is:
'''

response = get_completion(prompt)
print(response)

## Chain-of-Though Prompting (CoT)

In [ ]:
# Example with zero-shot
prompt='''
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Example with few-shot prompting
prompt= '''
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Example with Chain-of-Thought prompting
prompt = '''
The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.
The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.
The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.
The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Zero-shot Chain-of-Though prompting

prompt='''
I went to the market and bought 10 apples.
I gave 2 apples to the neighbor and 2 to the repairman.
I then went and bought 5 more apples and ate 1.
How many apples did I remain with?
Let's think step by step.
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Bài tập: sử dụng chiến lược self-consistency prompt để giải quyết bài toán sau
prompt='''
Tìm tất cả các cặp 3 số có tổng là 26
(1, 5, 10, 20, 6, 8, 12)
'''

response = get_completion(prompt)
print(response)

## Automatic Chain-of-Thought

When applying chain-of-thought prompting with demonstrations, the process involves hand-crafting effective and diverse examples. This manual effort could lead to suboptimal solutions. [Zhang et al. (2022)](https://arxiv.org/abs/2210.03493) propose an approach to eliminate manual efforts by leveraging LLMs with "Let's think step by step" prompt to generate reasoning chains for demonstrations one by one. This automatic process can still end up with mistakes in generated chains. To mitigate the effects of the mistakes, the diversity of demonstrations matter This work proposes Auto-CoT, which samples questions with diversity and generates reasoning chains to construct the demonstrations.

Auto-CoT consists of two main stages:
- Stage 1): question clustering: partition questions of a given dataset into a few clusters
- Stage 2): demonstration sampling: select a representative question from each cluster and generate its reasoning chain using Zero-Shot-CoT with simple heuristics

The simple heuristics could be length of questions (e.g., 60 tokens) and number of steps in rationale (e.g., 5 reasoning steps). This encourages the model to use simple and accurate demonstrations.

<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fauto-cot.642d9bad.png&w=1200&q=75" alt="drawing" style="width:200px;"/>

In [ ]:
!git clone https://github.com/amazon-science/auto-cot.git
%cd auto-cot
!pip install -r requirements.txt
import sys
sys.argv=['']
del sys
from api import cot
question = "In a video game, each enemy defeated gives you 7 points. If a level has 11 enemies total and you destroy all but 8 of them, how many points would you earn?"
print("Example: Auto-CoT")
cot(method="auto_cot", question=question)

## Meta Prompting

In [ ]:
prompt = '''
Problem Statement:
Problem: [question to be answered]
Solution Structure:
1. Begin the response with "Let's think step by step."
2. Follow with the reasoning steps, ensuring the solution process is broken down clearly and
logically.
3. End the solution with the final answer encapsulated in a LaTeX-formatted box, -, for clarity
and emphasis.
4. Finally, state "The answer is [final answer to the problem].", with the final answer presented in
LaTeX notation.
'''

response = get_completion(prompt)
print(response)

In [ ]:
prompt= '''
Problem: Find the domain of the expression va-2.
Solution: The expressions inside each square root must be non-negative. Therefore, x - 2 ≥ 0, so x ≥ 2, and 5 - x ≥ 0, so x ≤ 5. Also, the denominator cannot be equal to zero, so 5 - x > 0, which gives x < 5. Therefore, the domain of the expression is [2,5) .
Final Answer: The final answer is 2, 5). I hope it is correct.
Problem: If det A = 2 and det B = 12, then find det (AB).
Solution: We have that det (AB) = (det A)(det B) = (2)(12) = 24 .
Final Answer: The final answer is 24. I hope it is correct.
'''

## Self-consistency

In [ ]:
prompt = '''
When I was 6 my sister was half my age. Now
I’m 70 how old is my sister?
'''

response = get_completion(prompt)
print(response)

In [ ]:
prompt='''
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent $15. She has $8 left.
Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Bài tập: sử dụng các chiến lược prompt đã học để giải quyết bài toán sau
prompt='''
 Giờ Văn cô giáo trả bài kiểm tra.
 Bốn bạn Tuấn, Hùng, Lan, Quân ngồi cùng bàn đều đạt điểm 8 trở lên.
 Giờ ra chơi Phương hỏi điểm của 4 bạn, Tuấn trả lời :
  - Lan không đạt điểm 10, mình và Quân không đạt điểm 9 còn Hùng không đạt điểm 8.
 Hùng thì nói :
  - Mình không đạt điểm 10, Lan không đạt điểm 9 còn Tuấn và Quân đều không đạt điểm 8.

Bạn hãy cho biết mỗi người đã đạt mấy điểm?.
'''

response = get_completion(prompt)
print(response)

## Generate Knowledge Prompting
<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fgen-knowledge.055b8d37.png&w=828&q=75" alt="drawing" style="width:180px;"/>

In [ ]:
# Normal prompt
prompt='''
Part of golf is trying to get a higher point total than others. Yes or No?
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Knowledge generate prompt
prompt='''
Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.
Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.
Input: A fish is capable of thinking.
Knowledge: Fish are more intelligent than they appear. In many areas, such as memory, their cognitive powers match or exceed those of ’higher’ vertebrates including non-human primates. Fish’s long-term memories help them keep track of complex social relationships.
Input: A common effect of smoking lots of cigarettes in one’s lifetime is a higher than normal chance of getting lung cancer.
Knowledge: Those who consistently averaged less than one cigarette per day over their lifetime had nine times the risk of dying from lung cancer than never smokers. Among people who smoked between one and 10 cigarettes per day, the risk of dying from lung cancer was nearly 12 times higher than that of never smokers.
Input: A rock is the same size as a pebble.
Knowledge: A pebble is a clast of rock with a particle size of 4 to 64 millimetres based on the Udden-Wentworth scale of sedimentology. Pebbles are generally considered larger than granules (2 to 4 millimetres diameter) and smaller than cobbles (64 to 256 millimetres diameter).
Input: Part of golf is trying to get a higher point total than others.
Knowledge:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Knowledge obtained from LLMs
knowledge1 = '''
In golf, the objective is actually to have the lowest score possible, not the highest. Players aim to hit the ball into the hole with the fewest number of strokes, as each stroke is counted as a point against them. The player with the lowest total score at the end of the game is the winner.
'''
knowledge2='''
While the ultimate goal in golf is to have the lowest score possible, the scoring system in golf is based on the number of strokes taken to complete each hole. The player with the lowest total number of strokes at the end of the round is the winner. So, in golf, the objective is to have the lowest score, not the highest.
'''

prompt=f'''
Question: Part of golf is trying to get a higher point total than others. Yes or No?
Knowledge: {knowledge1}
Knowledge: {knowledge2}
Explain and Answer:
'''

response = get_completion(prompt)
print(response)

In [ ]:
# Bài tập: thực hiện các chiến lược generate knowledge prompt để giải quyết câu hỏi sau:
prompt='''
chiều cao trung bình của cầu thủ bóng đá cao hơn một cầu thủ bóng rổ. Đúng hay sai ?
'''

response = get_completion(prompt)
print(response)

## Tree-of-Thought
<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FTOT.3b13bc5e.png&w=1200&q=75" alt="drawing" style="width:180px;"/>
